# Web Scrapping

Cristobal Donoso Oliva<br>
Universidad de Concepcion, Chile<br>
cridonoso@inf.udec.cl<br>


_Viernes 16 de Octubre del 2020_

**Web scrapping** es una tecnica informatica la cual nos permite extraer datos desde paginas web. Podemos separar las tecnicas dependiendo el contenido de la pagina:
- contenido estatico: Paginas HTML fijas
- contenido dinamico: Paginas interactivas HTML + javascript

<br>
Los sofware de scrapping simulan ser un ser humano extrayendo los datos que se encuentran de manera visible en la pagina. La automatizacion de este proceso nos permite extraer grandes volumenes de datos de manera rapida y precisa. 

### Librerias en Python

Existen varias librerias en Python para realizar scrapping. A continuacion una tabla comparativa de los frameworks mas populares en python (*Fuente: [Python Web Scrapping - Kite, 2020](https://www.youtube.com/watch?v=zucvHSQsKHA)*)

 <img src="./images/0.png" alt="Drawing" style="width: 600px;"/> 

En este tutorial utilizaremos **[BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)**

In [1]:
!pip install requests # para cargar URLs
!pip install bs4

In [2]:
import requests 
from bs4 import BeautifulSoup

Lo primero que debemos hacer es **cargar el codigo html** desde la URL de la pagina de interes.
<br><br>
En este tutorial accederemos a las **ultimas publicaciones** del [arXiv](https://arxiv.org/)

Fijense que la consulta del buscador esta explicita en la URL por lo que podriamos modificar esta directamente 
<br><br>
```https://arxiv.org/search/?query=machine+learning&searchtype=all&abstracts=show&order=-announced_date_first&size=200```
<br><br>
en este caso, podemos crear una variable de tipo ```<list>``` para que reciba las palabras claves a buscar.

In [3]:
key_words = ['machine', 'learning']

str_keywords = '+'.join(key_words)

base = 'https://arxiv.org/search/?query={}&searchtype=all&abstracts=show&order=-announced_date_first&size=200'
url  = base.format(str_keywords)
print(url)

https://arxiv.org/search/?query=machine+learning&searchtype=all&abstracts=show&order=-announced_date_first&size=200


Una vez que tenemos nuestro link, podemos solicitar el codigo HTML de la pagina

In [4]:
%%time
html_page = requests.get(url) 

CPU times: user 26.4 ms, sys: 11.9 ms, total: 38.3 ms
Wall time: 5.56 s


Una vez cargado el texto plano HTML debemos instanciar un Parser desde BeautifulSoup. Un parser es un **analizador sintactico**. En este caso BeautifulSoup nos permite trabajar con diferentes sintaxis, tales como HTML o XML

In [5]:
soup = BeautifulSoup(html_page.text, 'html.parser') # Instanciamos nuestro scrapper

Ahora debemos extraer la informacion que necesitamos. La forma mas directa de hacerlo es desde el navegador:
1. Abre la pagina web
2. Click derecho sobre el item de interes  <img src="./images/1.png" alt="Drawing" style="width: 600px;"/> 
3. Click sobre inspeccionar <img src="./images/2.png" alt="Drawing" style="width: 200px;"/> 
4. Revisa las etiquetas HTML para acceder al contenido en nuestro scrapper <img src="./images/3.png" alt="Drawing" style="width: 600px;"/> 

En este ejemplo sacaremos los titulos de los papers, por lo tanto debemos extraer los parrafos ```<p>``` cuya clase tenga el nombre ```class="title is-5 mathjax">```

In [7]:
titulos = soup.find_all('p', attrs = {"class": 'title is-5 mathjax'})

La funcion ```find_all()``` retorna una lista de tipo ```bs4.element.Tag``` con las coincidencias de nuestra consulta. 

In [16]:
for titulo in titulos:
    print(titulo.text.strip())
    break

Online and Distribution-Free Robustness: Regression and Contextual Bandits with Huber Contamination


Y ya esta! 

### Ejercicio: Obtieniendo los links de los papers 

En este caso vamos a leer el contenedor HTML de manera un poco mas general. Segun nuestra consulta previa, todos los resultados vienen encapsulados en el tag ```<li>``` cuya clase es ```class="arxiv-result"```

In [17]:
resultados = soup.find_all('li', attrs = {"class": 'arxiv-result'})

In [47]:
titulos_list  = [] # Para guardar los titulos
pdf_link_list = [] # Para guardar los links

for res in resultados:
    titulo = res.find('p', attrs = {"class": "title is-5 mathjax"}) # Accedemos al tag del titulo
    link_content = res.find('p', attrs = {"class": "list-title is-inline-block"}) # accedemos al tag de los link
    link_content = link_content.find('span') # luego al tag de span dentro del tag anterior
    link_content = link_content.find_all('a') # finalmente extraemos todos los enlaces <a> dentro de ese tag
    links = [link.attrs.get('href') for link in link_content] # iteramos y extraemos el atributo href
    
    # Guardamos el titulo del paper 
    titulos_list.append(titulo.text.strip())
    
    # El primer link siempre sera pdf 
    pdf_link_list.append(links[0])

Ahora que tenemos todo guardado podemos crear un Frame de Pandas para ordenar los resultados

In [48]:
import pandas as pd

In [63]:
arxiv_hoy = pd.DataFrame() # Inicializamos un Frame vacio
arxiv_hoy['Titulo'] = titulos_list # Creamos columna con los titulos
arxiv_hoy['Link'] = pdf_link_list  # Creamos columna con los links al pdf 

In [64]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

arxiv_hoy.style.format({'Link': make_clickable})

,Titulo,Link
0,Online and Distribution-Free Robustness: Regression and Contextual Bandits with Huber Contamination,https://arxiv.org/pdf/2010.04157
1,Automating Inference of Binary Microlensing Events with Neural Density Estimation,https://arxiv.org/pdf/2010.04156
2,Strongly lensed supernovae as a self-sufficient probe of the distance duality relation,https://arxiv.org/pdf/2010.04155
3,Olympus: a benchmarking framework for noisy optimization and experiment planning,https://arxiv.org/pdf/2010.04153
4,Automatic generation of reviews of scientific papers,https://arxiv.org/pdf/2010.04147
5,Leakage-Adjusted Simulatability: Can Models Generate Non-Trivial Explanations of Their Behavior in Natural Language?,https://arxiv.org/pdf/2010.04119
6,Interlocking Backpropagation: Improving depthwise model-parallelism,https://arxiv.org/pdf/2010.04116
7,A Machine Learning Framework for Computing the Most Probable Paths of Stochastic Dynamical Systems,https://arxiv.org/pdf/2010.04114
8,Information-Driven Adaptive Sensing Based on Deep Reinforcement Learning,https://arxiv.org/pdf/2010.04112
9,Set Prediction without Imposing Structure as Conditional Density Estimation,https://arxiv.org/pdf/2010.04109
